# Day 3: Visualizing time series data

### Importing Libraries


In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import os

import plotly.io as pio
pio.templates.default = "ggplot2"

### Get data

In [12]:
gold = pd.read_csv(os.path.join("..", "data", "gold_prices.csv"), index_col=["Date"], parse_dates=True)

gold.head()

,Price
Date,
2016-01-11,104.739998
2016-01-12,104.209999
2016-01-13,104.720001
2016-01-14,103.019997
2016-01-15,104.080002


## Visualizatons

### Importing Libraries

In [13]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go 

In [14]:
# Preprocessing

gold['Price'] = gold['Price'].astype(float)
gold['Price'] = gold['Price'] * 10.8
gold['Price'] = gold['Price'].round(0)

### Line Plots

In [15]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [17]:
fig1 = px.line(
    gold.reset_index(), 
    x='Date',
    y='Price', 
    title='Gold (GLD) Price: Jan 2016 - Jan 2026',
    labels={'Price': 'Price (USD)', 'Date': 'Date'}
)
fig1.update_yaxes(tickformat=".0f", dtick=200, range=[1000, 5000])
fig1.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
fig1.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig1.show()

# 3. Rolling Statistics Visualization
# 30-day and 90-day Moving Averages
gold['MA30'] = gold['Price'].rolling(window=30).mean()
gold['MA90'] = gold['Price'].rolling(window=90).mean()
gold['Std30'] = gold['Price'].rolling(window=30).std()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=gold.index, y=gold['Price'], name='Price', opacity=0.5))
fig2.add_trace(go.Scatter(x=gold.index, y=gold['MA30'], name='30-Day MA', line=dict(color='orange')))
fig2.add_trace(go.Scatter(x=gold.index, y=gold['MA90'], name='90-Day MA', line=dict(color='red')))
fig2.update_layout(title='Gold Price with Rolling Averages', xaxis_title='Date', yaxis_title='Price (USD)')
fig2.show()

# 4. Seasonal Decomposition
# We resample to monthly to make the decomposition clearer
gold_monthly = gold['Price'].resample('ME').mean()
result = seasonal_decompose(gold_monthly, model='additive', period=12)

# Create subplots for decomposition manually to use Plotly
from plotly.subplots import make_subplots
fig3 = make_subplots(rows=4, cols=1, subplot_titles=("Observed", "Trend", "Seasonal", "Residuals"))

fig3.add_trace(go.Scatter(x=gold_monthly.index, y=result.observed, name='Observed'), row=1, col=1)
fig3.add_trace(go.Scatter(x=gold_monthly.index, y=result.trend, name='Trend'), row=2, col=1)
fig3.add_trace(go.Scatter(x=gold_monthly.index, y=result.seasonal, name='Seasonal'), row=3, col=1)
fig3.add_trace(go.Scatter(x=gold_monthly.index, y=result.resid, name='Residuals'), row=4, col=1)

fig3.update_layout(height=800, title_text="Gold Price Seasonal Decomposition (Monthly)", showlegend=False)
fig3.show()